In [1]:
import sys
!{sys.executable} -m pip install pulp

import pulp 


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# ------------------------
# 1. Dane
# ------------------------

dostawcy = ['R', 'W', 'B', 'T']
odbiorcy = ['S', 'D', 'C', 'P']
pojazdy = ['8', '20']

# Koszty jednego pustego przebiegu (Tabela 9)
C = {
    # 8-tonowe
    ('R','S','8'): 338.77, ('R','D','8'): 236.75, ('R','C','8'): 545.44, ('R','P','8'): 485.00,
    ('W','S','8'): 580.75, ('W','D','8'): 438.18, ('W','C','8'): 423.79, ('W','P','8'): 557.00,
    ('B','S','8'): 323.08, ('B','D','8'): 349.24, ('B','C','8'): 542.82, ('B','P','8'): 483.00,
    ('T','S','8'): 279.91, ('T','D','8'): 306.07, ('T','C','8'): 499.66, ('T','P','8'): 446.00,
    # 20-tonowe
    ('R','S','20'): 552.19, ('R','D','20'): 385.89, ('R','C','20'): 889.04, ('R','P','20'): 790.00,
    ('W','S','20'): 946.61, ('W','D','20'): 714.22, ('W','C','20'): 690.77, ('W','P','20'): 908.00,
    ('B','S','20'): 526.60, ('B','D','20'): 569.24, ('B','C','20'): 884.78, ('B','P','20'): 788.00,
    ('T','S','20'): 456.25, ('T','D','20'): 498.89, ('T','C','20'): 814.42, ('T','P','20'): 727.00
}

# Nadwyżki (dostawcy)
nadwyzki = {
    'R': {'8': 11, '20': 4},
    'W': {'8': 1,  '20': 3},
    'B': {'8': 12, '20': 9},
    'T': {'8': 7,  '20': 5}
}

# Niedobory (odbiorcy)
niedobory = {
    'S': {'8': 11, '20': 8},
    'D': {'8': 12, '20': 6},
    'C': {'8': 3,  '20': 4},
    'P': {'8': 5,  '20': 3}
}

# ------------------------
# 2. Definicja zmiennych decyzyjnych
# ------------------------

x = pulp.LpVariable.dicts("x",
                          [(i,j,k) for i in dostawcy for j in odbiorcy for k in pojazdy],
                          lowBound=0, cat='Integer')

# ------------------------
# 3. Definicja problemu
# ------------------------

prob = pulp.LpProblem("Minimalizacja_pustych_przebiegow", pulp.LpMinimize)

# Funkcja celu: minimalizacja kosztów
prob += pulp.lpSum([C[(i,j,k)] * x[(i,j,k)] for i in dostawcy for j in odbiorcy for k in pojazdy])

# ------------------------
# 4. Ograniczenia
# ------------------------

# a) Dostawcy: całkowita liczba pojazdów wychodzących = nadwyżka
for i in dostawcy:
    for k in pojazdy:
        prob += pulp.lpSum([x[(i,j,k)] for j in odbiorcy]) == nadwyzki[i][k]

# b) Odbiorcy: całkowita liczba pojazdów docierających = niedobór
for j in odbiorcy:
    for k in pojazdy:
        prob += pulp.lpSum([x[(i,j,k)] for i in dostawcy]) == niedobory[j][k]

# ------------------------
# 5. Rozwiązanie
# ------------------------

prob.solve()

# ------------------------
# 6. Wyniki
# ------------------------

print(f"Status rozwiązania: {pulp.LpStatus[prob.status]}")
print(f"Minimalny koszt całkowity: {pulp.value(prob.objective):.2f} zł\n")

print("Optymalna liczba pustych przebiegów (8-tonowe):")
for i in dostawcy:
    for j in odbiorcy:
        print(f"{i} -> {j} : {x[(i,j,'8')].varValue}")

print("\nOptymalna liczba pustych przebiegów (20-tonowe):")
for i in dostawcy:
    for j in odbiorcy:
        print(f"{i} -> {j} : {x[(i,j,'20')].varValue}")


Status rozwiązania: Optimal
Minimalny koszt całkowity: 21993.78 zł

Optymalna liczba pustych przebiegów (8-tonowe):
R -> S : 0.0
R -> D : 11.0
R -> C : 0.0
R -> P : 0.0
W -> S : 0.0
W -> D : 0.0
W -> C : 1.0
W -> P : 0.0
B -> S : 5.0
B -> D : 0.0
B -> C : 2.0
B -> P : 5.0
T -> S : 6.0
T -> D : 1.0
T -> C : 0.0
T -> P : 0.0

Optymalna liczba pustych przebiegów (20-tonowe):
R -> S : 0.0
R -> D : 4.0
R -> C : 0.0
R -> P : 0.0
W -> S : 0.0
W -> D : 0.0
W -> C : 3.0
W -> P : 0.0
B -> S : 6.0
B -> D : 0.0
B -> C : 0.0
B -> P : 3.0
T -> S : 2.0
T -> D : 2.0
T -> C : 1.0
T -> P : 0.0
